## <center>Создание спам-фильтра для СМС-сообщений с использованием наивного байесовского классификатора: Построение предиктивного алгоритма с нуля</center>

                                                           Проект выполнен: Резвухин Д.И., январь 2023 г.

### Введение

**Наивный байесовский классификатор** — простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими (наивными) предположениями о независимости признаков классифицируемых объектов. В зависимости от точной природы вероятностной модели, наивные байесовские классификаторы могут обучаться очень эффективно, несмотря на малореалистичное предположение о независимости признаков. Достоинством классификатора также является малое количество данных, необходимых для обучения, оценки параметров и классификации. 

В ходе реализации данного проекта мы будем использовать мультиномиальный наивный байесовский классификатор, чтобы создать спам-фильтр, то есть обучить компьютер классифицировать сообщения на две категории: "спам" (spam) и "не спам" (ham). При выполнении задач будет использоваться датасет из 5572 смс-сообщений, которые уже были корректно классифицированы человеком. Датасет собран исследователями Tiago A. Almeida и José María Gómez Hidalgo, и находится в свободном доступе на сайте [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). Изначально он использовался для изучения проблемы спама в смс-сообщениях и сравнения различных ML-алгоритмов в качестве фильтров, результаты были опубликованы в работе Almeida, T.A., Gomez Hidalgo, J.M., Yamakami, A. Contributions to the Study of SMS Spam Filtering: New Collection and Results. Proceedings of the 2011 ACM Symposium on Document Engineering (DOCENG'11), Mountain View, CA, USA, 2011.

### 1. Импортирование библиотек и знакомство с датасетом

Импортируем библиотеку `pandas` и посмотрим сводную информацию о датасете. 

In [1]:
"""В датасете нет заголовка, 
   поэтому отметим это в принимаемых функцией `pd.read_csv` аргументах 
   и дадим столбцам названия."""

import pandas as pd

sms_data = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
sms_data.info()
sms_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Датасет имеет очень простую структуру. В столбце `Label` всего два значения: 'spam' и 'ham', что соответсвует спаму и обычным сообщениям от абонентов (ham – антоним к слову spam). В ячейках столбца `SMS` находится текст смс-сообщений.

Посчитаем процентное соотношение спама к общему количеству смс-сообщений:

In [2]:
sms_data['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Таким образом, около 13% сообщений в датасете являются спамом. Оставшиеся 87% классифицированы человеком как обычные пользовательские сообщения (не спам).

### 2. Создание обучающей и тестовой выборок

Перед созданием непосредственно спам-фильтра, имеет смысл сначала подумать о способе проверки его эффективности. При разработке ПО (в том числе и спам-фильтра) создание релевантного теста должно предшествовать созданию самого программного обеспечения. Это необходимо для того, чтобы исключить возможность создания предвзятой проверки, которую алгоритм заведомо пройдет.

Мы разделим датасет на две части. 80% данных пойдут в обучающую выборку, а оставшиеся 20% — в тестовую. Такое разделение обусловлено тем, что для обучения алгоритма необходимо как можно больше доступных данных, но в то же время необходимо оставить часть данных для тестирования. Суть тестовой выборки заключается в том, что смс-сообщения в ней уже корректно классифицированы человеком как "спам" или "не спам". Применив к ним получившийся спам-фильтр и сравнив результаты, можно будет оценить его эффективность.

Разделим датасет на две части согласно пропорции 80/20. Суммарно в датасете представлено 5572 сообщения, поэтому в обучающую выборку пойдут 4457 смсок, а в тестовую — 1115. Проверим, что пропорция спама в ~13.4% сохранилась в обеих частях.

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sms_data, test_size=0.2, random_state=363563, stratify=sms_data['Label'])
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

spam_pct_train = (train['Label'] == 'spam').mean() * 100
spam_pct_test = (test['Label'] == 'spam').mean() * 100
print(f'Proportion of spam in the training set is {spam_pct_train:.2f}%')
print(f'Proportion of spam in the test set is {spam_pct_test:.2f}%')

Proportion of spam in the training set is 13.42%
Proportion of spam in the test set is 13.36%


### 3. Немного дата-клининга, пунктуация и регистр

Перед тем, как начать непосредственно расчет вероятностей, нам нужно привести данные в обучающей выборке к удобному формату, чтобы впоследствии можно было легко извлечь всю необходимую информацию. А именно, имеет смысл поменять структуру датасета следующим образом:

* оставить столбец `Label`
* столбец `SMS` разделить на ряд столбцов, в каждом из которых будет указана частота встречаемости слов в каждом сообщении
* каждый ряд при этом продолжит соответствовать одному сообщению. 

Например, предположим, что сообщение "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!" разделится на столбцы с данными `spam, 2, 2, 1, 1, 0, ..., 0`. Это будет означать, что слово "secret" в этом сообщении встречено 2 раза, слово "prize" также встречается два раза, слова "claim" и "now" встречаются по одному разу, а остальных слов из списка уникальных слов в этом сообщении нет. Таким образом, мы должны превратить исходный датафрейм такого типа:

|   | Label | SMS                                    |
|---|-------|----------------------------------------|
| 0 | spam  | SECRET PRIZE! CLAIM SECRET PRIZE NOW!! |
| 1 | ham   | Coming to my secret party?             |
| 2 | spam  | Winner! Claim secret prize now!        |

В новый датафрейм, который будет содержать таблицу встречаемости слов:

|   | Label | secret | prize | claim | now | coming | to | my | party | winner |
|---|-------|--------|-------|-------|-----|--------|----|----|-------|--------|
| 0 | spam  | 2      | 2     | 1     | 1   | 0      | 0  | 0  | 0     | 0      |
| 1 | ham   | 1      | 0     | 0     | 0   | 1      | 1  | 1  | 1     | 0      |
| 2 | spam  | 1      | 1     | 1     | 1   | 0      | 0  | 0  | 0     | 1      |

Помимо этого, нужно учесть, что:
* все слова в списке уникальных услов будут представлены в нижнем регистре (например, "SECRET" и "secret" соответствуют одному слову)
* важны только слова, пунктуация значения не имеет

На данном шаге начнем модификацию обучающей выборки с удаления знаков препинания и переводу всех букв к нижнему регистру. Используем простой RegEx шаблон, чтобы найти все символы, которые не являются буквами, и заменим их на пробел.

In [4]:
train['SMS'] = train['SMS'].str.replace('\W', ' ', regex=True).str.lower()
train['SMS'].head()

0                          tell me pa  how is pain de 
1    camera   you are awarded a sipix digital camer...
2    oh ok i didnt know what you meant  yep i am ba...
3    free ringtone  reply real or poly eg real1 1  ...
4    hi  wk been ok   on hols now  yes on for a bit...
Name: SMS, dtype: object

### 4. Создание списка уникальных слов

Для того, чтобы провести обучение классификатора, нужно создать список уникальных слов, встречающихся в сообщениях (vocabulary). Каждое слово в этом списке должно иметь строковый тип данных (`str`). Для этого разделим каждое сообщение в обучающей выборке на список слов и потом каждое слово занесем в инициализированный заранее пустой список:

In [5]:
train['SMS'] = train['SMS'].str.split()

vocabulary = []

for sms_split in train['SMS']:
    for word in sms_split:
        vocabulary.append(word)
        
# избавимся от повторов, преобразовав список в множество
vocabulary = list(set(vocabulary))
print(f'Number of unique words in the vocabulary: {len(vocabulary)}')

Number of unique words in the vocabulary: 7878


### 5. Завершение модификации обучающей выборки

Теперь, используя список уникальных слов в сообщениях, мы можем видоизменить датафрейм обучающей выборки таким образом, чтобы он соответствовал плану из пункта 3. 

In [6]:
"""Для этого создадим новый датафрейм из словаря, 
   в котором каждый ключ будет уникальным словом из списка слов, 
   а значение будет представлять собой список с длиной, 
   равной количеству смс-сообщений в обучающей выборке. 
   
   Каждый элемент такого списка будет изначально равен нулю. 
   Значение будет меняться (последовательно повышаться на 1) для тех слов, 
   которые встречаются в конкретном сообщении.

   Мы также соединим полученный датафрейм с изначальной обучающей выборкой, 
   чтобы добавить в него столбцы `Label` и `SMS`."""

word_counts_per_sms = {word: [0] * len(train['SMS']) for word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)
train_clean = pd.concat([train, word_counts_per_sms], axis=1)
train_clean.tail()

,Label,SMS,settle,kvb,b,disastrous,banter,09058094507,incorrect,09056242159,...,asda,depressed,bruv,announced,pc,complaint,seen,645,lightly,cos
4452,spam,"[this, is, the, 2nd, time, we, have, tried, 2,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4453,ham,"[u, gd, lor, go, shopping, i, got, stuff, to, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,spam,"[gsoh, good, with, spam, the, ladies, u, could...",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,ham,"[i, dont, want, to, hear, anything]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,ham,"[s, this, will, increase, the, chance, of, win...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Немного теории

После того, как мы подготовили данные для задач классификации, можно начать создание спам-фильтра. Наивный байесовский алгоритм классифицирует сообщение на основе сравнения двух апостериорных вероятностей, которые получаются при вычислении следующих выражений:
\begin{equation}
P(Spam|\psi_1, \psi_2,...,\psi_n) \propto P(Spam) \cdot \prod_{i=1}^n P(\psi_{i}|Spam)
\end{equation}

\begin{equation}
P(Ham|\psi_1, \psi_2,...,\psi_n) \propto P(Ham) \cdot \prod_{i=1}^n P(\psi_{i}|Ham)
\end{equation}

где знак $\propto$ означает пропорциональность, $\psi$ обозначает частоты встречаемости отдельных слов в сообщениях, а  $\prod_{i=1}^n$ означает произведение (нам нужно перемножить значения $P(\psi_{i}|Spam)$ для отдельных слов). 

Если значение первого выражения больше второго, то сообщение классифицируется как спам. Фактически, данные выражения представляют собой обычную формулу Байеса, но без знаменателя, который для этих выражений одинаков. Такое упрощение возможно по причине того, что нас интересует сравнение вероятностей (больше-меньше), а не сами значения вероятностей. Однако из-за того, что знаменатель сокращен, в выражениях используется не знак равенства, а знак пропорциональности $\propto$.

Чтобы рассчитать $P(\psi_{i}|Spam)$ и $P(\psi_{i}|Spam)$ из формул выше, нам также потребуются следующие уравнения:

\begin{equation}
P(\psi_i|Spam) = \frac{N_{\psi_i|Spam} + \alpha}{N_{Spam}+ \alpha \cdot N_{vocabulary}}
\end{equation}

\begin{equation}
P(\psi_i|Ham) = \frac{N_{\psi_i|Ham} + \alpha}{N_{Ham}+ \alpha \cdot N_{vocabulary}}
\end{equation}

В этих формулах:
* $N_{\psi_i|Spam}$ — частота встречамости $\psi_i$ слова в спаме
* $N_{Spam}$ — общее количество слов в спаме
* $N_{vocabulary}$ — общее количество уникальных слов
* $\alpha = 1$ — аддитивное сглаживание (сглаживание Лапласа). 

Данные уравнения позволяют вычислить частоты встречаемости отдельных слов в категориях "Спам" и "Не спам" (для этого мы ранее производили манипуляции с датафреймом). Поскольку возможны ситуации, когда все выражение (1) или (2) обнуляется из-за того, что одно из слов не нашлось в категории Spam или Ham (т.е. множитель будет равен нулю), то используется аддитивное сглаживание – параметр $\alpha$, который делает значение множителя очень низким, однако не равным нулю.

### 7. Вычисление констант

Отметим, что некоторые составляющие части уравнений имеет одно и то же значение вне завимости от классифицируемого сообщения. Рассчитаем такие константы как априорные вероятности $P(Spam)$ и $P(Ham)$, а также $N_{Spam}$, $N_{Ham}$ и $N_{Vocabulary}$.

$P(Spam)$ и $P(Ham)$ представляют собой пропорции "спама" и "не спама", соответственно, относительно числа сообщений в целом в обучающей выборке. По сути, мы уже считали эти вероятности, когда разделяли датасет на выборки.

In [7]:
train_spam = train_clean[train_clean['Label'] == 'spam']
train_ham = train_clean[train_clean['Label'] == 'ham']

# вычисление P(Spam) и P(Ham), априорные вероятности
p_spam = len(train_spam) / len(train)
p_ham = len(train_ham) / len(train)
print(f'P(Spam) is {p_spam:.3f}')
print(f'P(Ham) is {p_ham:.3f}')

# вычисление N(Spam) и N(Ham), общее количество слов той и другой категории
n_spam = train_spam['SMS'].apply(len).sum()
n_ham = train_ham['SMS'].apply(len).sum()
print(f'Number of words in Spam, N(Spam) is {n_spam}')
print(f'Number of words in Ham, N(Ham) is {n_ham}')

# количество уникальных слов в списке уникальных слов
n_voc = len(vocabulary)
print(f'Total number of unique words is {n_voc}')

# сглаживание Лапласа
alpha = 1

P(Spam) is 0.134
P(Ham) is 0.866
Number of words in Spam, N(Spam) is 15243
Number of words in Ham, N(Ham) is 57527
Total number of unique words is 7878


### 8. Вычисление остальных параметров 

В формулах, приведенных выше, параметры $P(\psi_{i}|Spam)$ и $P(\psi_{i}|Spam)$ будут разными в зависимости от слова. При этом, для каждого слова значения этих параметров будут постоянными вне зависимости от сообщения. Например, представим, что нами получено два сообщения, содержащие слово "secret":

* "secret code"
* "secret party 2night"

Для обоих сообщений нужно рассчитать значение параметра $P(secret|Spam)$. Это можно сделать по следующей формуле:

\begin{equation}
P(secret|Spam) = \frac{N_{secret|Spam} + \alpha}{N_{Spam}+ \alpha \cdot N_{vocabulary}}
\end{equation}

Для обоих сообщений значение этого параметра будет одинаковым, равно как и для любого другого сообщения, содержащего слово "secret". Ключевая особенность здесь заключается в том, что значение $P(secret|Spam)$ зависит только от обучающей выборки, и поэтому в рамках выборки будет постоянным. Это означает, что мы можем использовать обучающую выборку, чтобы рассчитать вероятности  $P(word|Spam)$ и $P(word|Ham)$ для каждого уникального слова. Таким образом, предварительный расчет большого количества значений делает байесовский классификатор очень быстрым. Когда встает необходимость классифицировать новое сообщение, большая часть вычислений уже сделана, что позволяет почти мгновенно провести работу по классификации.

Посчитаем значения $P(word|Spam)$ и $P(word|Ham)$ для каждого уникального слова. 

In [8]:
"""Для этого создадим два словаря, в которых ключи будут уникальными словами, 
   а значения изначально будут представлены в виде нулей. 
   После этого для каждого уникального слова рассчитаем значения P(word|Spam) и P(word|Ham) 
   и заменим нули в словарях на полученные значения."""

train_spam_freq = {word: 0 for word in vocabulary}
train_ham_freq = {word: 0 for word in vocabulary}

for word in vocabulary:
    p_word_spam = (train_spam[word].sum() + alpha) / (n_spam + n_voc)
    p_word_ham = (train_ham[word].sum() + alpha) / (n_ham + n_voc)
    train_spam_freq[word] = p_word_spam
    train_ham_freq[word] = p_word_ham

### 9. Классификация нового сообщения

Проведя большую подготовительную работу, мы можем приступить к созданию ядра спам-фильтра. Спам-фильтр можно представить в виде функции, которая:

* принимает в качестве аргумента сообщение, состоящее из некоторого количества слов ($\psi_1, \psi_2,...,\psi_n$)
* рассчитывает значения $P(Spam|\psi_1, \psi_2,...,\psi_n)$ и $P(Ham|\psi_1, \psi_2,...,\psi_n)$ по формулам
* сравнивает эти значения

Если $P(Spam|\psi_1, \psi_2,...,\psi_n)$ > $P(Ham|\psi_1, \psi_2,...,\psi_n)$, то сообщение классифицируется как спам, и наоборот. В случае абсолютно равных  вероятностей, алгоритм может опционально указать, что требуется помощь человека. Мы не будем использовать округление, поэтому такой вариант практически невозможен. 

Напишем, функцию, представляющую собой спам-фильтр:

In [9]:
import re

def classify(message):
    """Classifies a message into Spam or Ham based on the Naive Bayes approach"""    
    message = re.sub('\W', ' ', message).lower().split()
    
    # инициализируем начальные (априорные) значения вероятностей для слов
    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham 
    
    for word in message:
        if word in train_spam_freq:
            p_spam_given_message *= train_spam_freq[word]
        
        if word in train_ham_freq:
            p_ham_given_message *= train_ham_freq[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message >= p_spam_given_message:
        print('Label: Ham')
    else:
        print('Label: Spam')


Попробуем применить функцию для классификации новых сообщений. Создадим одно из них так, чтобы оно напоминало спам, а второе, наоборот, походило на обычное сообщение от одного юзера к другому:

In [10]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')
print()
classify("Sounds good, Dmitriy, then see u there")

P(Spam|message): 1.6125728602444621e-25
P(Ham|message): 2.849010892846109e-27
Label: Spam

P(Spam|message): 2.353362543044702e-21
P(Ham|message): 7.987965872837592e-17
Label: Ham


Как можно убедиться, функция корректно классифицировала оба сообщения.

### 10. Оценка точности классификации

Теперь нам необходимо провести более подробный тестовый анализ, чтобы посмотреть, как классификатор ведет себя на тестовой выборке, т.е. насколько корректно он классифицирует самые разные сообщения. В процессе обучения алгоритм не встречался с сообщениями из тестовой выборки, поэтому для него они будут новыми.

Немного видоизменим функцию `classify`, чтобы она возвращала результат классификации (без print). 

In [11]:
def classify_test(message):
    """Classifies a message into Spam or Ham based on the Naive Bayes approach"""
    message = re.sub('\W', ' ', message).lower().split()
    
    # инициализируем начальные (априорные) значения вероятностей для слов
    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham 
    
    for word in message:
        if word in train_spam_freq:
            p_spam_given_message *= train_spam_freq[word]
        
        if word in train_ham_freq:
            p_ham_given_message *= train_ham_freq[word]

    if p_ham_given_message >= p_spam_given_message:
        return 'ham'
    else:
        return 'spam'


Теперь мы можем последовательно применить эту функцию ко всем сообщениям в тестовой выборке, а результаты классификации сохранить в отдельном (новом) столбце.

In [12]:
test['predicted'] = test['SMS'].apply(classify_test)
test.head(10)

,Label,SMS,predicted
0,ham,Where in abj are you serving. Are you staying ...,ham
1,ham,It has everything to do with the weather. Keep...,ham
2,ham,I got lousy sleep. I kept waking up every 2 ho...,ham
3,ham,I'm still pretty weak today .. Bad day ?,ham
4,ham,"Kate jackson rec center before 7ish, right?",ham
5,spam,Got what it takes 2 take part in the WRC Rally...,spam
6,ham,Still chance there. If you search hard you wil...,ham
7,spam,"Hi this is Amy, we will be sending you a free ...",ham
8,ham,What you doing?how are you?,ham
9,spam,Free 1st week entry 2 TEXTPOD 4 a chance 2 win...,spam


Теперь сравним классификацию с известным результатом (напомним, сообщения были заранее классифицированы человеком) и посчитаем саму метрику. Это будет количество правильных классификаций, поделенное на общее количество сообщений в выборке. В данном и похожем случаях, среднее от набора данных с логическим (булевым) типом будет как раз искомым результатом.

In [13]:
accuracy = (test['Label'] == test['predicted']).mean() * 100
print(f'Accuracy of this Naive Bayes spam-filter is {accuracy:.2f}%')

Accuracy of this Naive Bayes spam-filter is 98.57%


Наш алгоритм корректно классифицировал 98.57% всех сообщений. Это является очень достойным результатом, учитывая, что мы не проводили каких-либо более тонких настроек. 

Однако у такой метрики как accuracy есть существенный недостаток. В случае существенного дисбаланса классов, даже неработающий классификатор может иметь высокое значение accuracy. Например, если в выборке всего 1% сообщений составляет спам, то классификатор, который вообще не умеет находить спам и все сообщения классифицирует как ham, тем не менее покажет высокое значение accuracy (99%). В данном случае null accuracy (то есть значение accuracy, если алгоритм будет предсказывать все сообщения как ham), составляет 87% (это пропорция ham в датасете). Увеличение accuracy до 98.57% говорит о том, что алгоритм работает, и работает хорошо.

Мы также можем оценить эффективность алгоритма, посчитав некоторые другие метрики, такие как precision, recall и F1-score, а также построить confusion matrix, в которой будут указано количество правильно классифицированных сообщений, а также ошибки False Positive (I рода) и False Negative (II рода).

In [14]:
from sklearn.metrics import classification_report
print("Classification Report:", end='\n\n')
print(classification_report(test["Label"], test['predicted']))

Classification Report:

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       966
        spam       0.96      0.93      0.95       149

    accuracy                           0.99      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test["Label"], test['predicted'])

print('Confusion matrix:', end='\n\n')
print(cm, end='\n')
print('\nTrue Positives(TP) = ', cm[1,1])
print('\nTrue Negatives(TN) = ', cm[0,0])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix:

[[960   6]
 [ 10 139]]

True Positives(TP) =  139

True Negatives(TN) =  960

False Positives(FP) =  6

False Negatives(FN) =  10


Precision (точность, positive predicted value) – это доля объектов, названными классификатором положительными и при этом действительно являющихся положительными. В данном случае можно определить precision как способность не отправлять нормальные сообщения в спам. Этот показатель для нашего алгоритма составляет 0.96.

Recall (полнота, отклик, для положительного класса чувствительность, sensitivity, true positive rate) – это то, какую долю объектов положительного класса из всех объектов положительного класса нашёл алгоритм. Recall можно определить как способность улавливать, "чувствовать" спам. В данном случае значение Recall составляет 0.93. 

F-score (F-мера) представляет собой гармоническое среднее между precision и recall, и определяется по формуле F1 = 2 * (precision * recall) / (precision + recall). В данном случае значение F-score составялет 0.95.

Мы можем найти сообщения, которые были классифицированы неправильно, и посмотреть, есть ли у них какие-либо особенности. Посмотрим, какие сообщения на самом деле не являются спамом, однако были классифицированы как спам:

In [16]:
pd.set_option('display.max_colwidth', None)
test[(test['Label'] == 'ham') & (test['predicted'] == 'spam')]

,Label,SMS,predicted
18,ham,"wiskey Brandy Rum Gin Beer Vodka Scotch Shampain Wine ""KUDI""yarasu dhina vaazhthukkal. ..",spam
591,ham,"We have sent JD for Customer Service cum Accounts Executive to ur mail id, For details contact us",spam
716,ham,"Hey...Great deal...Farm tour 9am to 5pm $95/pax, $50 deposit by 16 May",spam
730,ham,Dhoni have luck to win some big title.so we will win:),spam
794,ham,Yavnt tried yet and never played original either,spam
992,ham,Waiting for your call.,spam


Интересно, что данные сообщения напоминают спам, так как являются достаточно короткими и содержат такие слова как "deal", "luck", "win", "call", "played". Недостаток спам-фильтра в том, что такие сообщения он будет воспринимать как спам. Но его можно понять:)

Теперь посмотрим сообщения, которые являются спамом, однако алгоритм классфифицировал их как не-спам:

In [17]:
test[(test['Label'] == 'spam') & (test['predicted'] == 'ham')]

,Label,SMS,predicted
7,spam,"Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...",ham
103,spam,thesmszone.com lets you send free anonymous and masked messages..im sending this message from there..do you see the potential for abuse???,ham
319,spam,RCT' THNQ Adrian for U text. Rgds Vatian,ham
465,spam,For sale - arsenal dartboard. Good condition but no doubles or trebles!,ham
884,spam,"Hi ya babe x u 4goten bout me?' scammers getting smart..Though this is a regular vodafone no, if you respond you get further prem rate msg/subscription. Other nos used also. Beware!",ham
963,spam,Block Breaker now comes in deluxe format with new features and great graphics from T-Mobile. Buy for just £5 by replying GET BBDELUXE and take the challenge,ham
1004,spam,Hi I'm sue. I am 20 years old and work as a lapdancer. I love sex. Text me live - I'm i my bedroom now. text SUE to 89555. By TextOperator G2 1DA 150ppmsg 18+,ham
1016,spam,"Hi babe its Chloe, how r u? I was smashed on saturday night, it was great! How was your weekend? U been missing me? SP visionsms.com Text stop to stop 150p/text",ham
1077,spam,"Xmas & New Years Eve tickets are now on sale from the club, during the day from 10am till 8pm, and on Thurs, Fri & Sat night this week. They're selling fast!",ham
1114,spam,This message is brought to you by GMW Ltd. and is not connected to the,ham


Как правило, это достаточно длинные сообщения, которые содержат слова с редкой встречаемостью. Также стоит отметить, что некоторые из этих сообщений маскируются под реальное сообщение от человека, что обманывает алгоритм.

### Заключение

В рамках данного проекта нам удалось создать достаточно точный спам-фильтр с использованием наивного байесовского классификатора. Алгоритм корректно классифицировал 98.6% сообщений в тестовой выборке, что гораздо выше, чем пропорция не-спам сообщений в датасете (~87%). Результаты вычисления таких метрик как accuracy, precision, recall и F-score, позволяют охарактеризовать спам-фильтр как достаточно надежный. Подавляющее большинство спам-сообщений такой фильтр не пропустит. В то же время, обычные не-спам сообщения также будут классифицированы правильно, за исключением некоторого количества коротких сообщений, которые похожи по нарративу на спам и включают такие слова как "deal", "luck", "win", "call", "play", "title" и т.д.